In [9]:
import torch
torch.utils.data.Subset

torch.utils.data.dataset.Subset

In [1]:
from dataset import MpalaTreeLiDAR
from torch import optim, nn
from torchvision import transforms
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm
import wandb

import pc_transform
from util import load_fake_dataset
from models.simpleview import SimpleView

# Initialize information about dataset and hyperparameters
ids, labels, classes = load_fake_dataset(min_points=500, max_points=600)
config = dict(
    learning_rate=0.001,
    momentum=0.9,
    epochs=100,
    batch_size=64,
    architecture='resnet18_4',
)

# Initialize dataset and dataloaders
dataset = MpalaTreeLiDAR(
    data_path='../data/MpalaForestGEO_LasClippedtoTreePolygons',
    ids=ids,
    labels=labels,
    classes=classes,
    transform=transforms.Compose([
        pc_transform.ToPointCloud(),
        pc_transform.ProjectPointCloud(),
        pc_transform.ExpandChannels(channels=1),
    ]),
)
trainset, testset = random_split(dataset, [0.85, 0.15])
trainloader = DataLoader(trainset, batch_size=config['batch_size'], shuffle=True)
print(type(trainset))
# Initialize model, criterion, optimizer
simpleview = SimpleView(num_views=6, num_classes=len(dataset.classes), architecture=config['architecture'])
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(simpleview.parameters(), lr=config['learning_rate'], momentum=config['momentum'])
print(type(criterion))
print(type(optimizer))
1/0
# Connect with w&b
wandb.init(
    project='my-test-project',
    config=config,
)
wandb.watch(simpleview)

# Train
for _ in tqdm(range(config['epochs'])):
    for x, y in trainloader:
        optimizer.zero_grad()
        outputs = simpleview(x)
        loss = criterion(outputs, y)
        loss.backward()
        optimizer.step()

        wandb.log({
            "train_loss": loss,
            "test_loss": test_loss,
            "train_acc": acc,
            "test_acc": test_acc,
        })

        break
    break

Found 16853 trees
Found only 705 trees with points in range [{min_points}, {max_points}]
<class 'torch.utils.data.dataset.Subset'>
<class 'torch.nn.modules.loss.CrossEntropyLoss'>
<class 'torch.optim.sgd.SGD'>


ZeroDivisionError: division by zero